# Orchestration Multi-Agents : Pipeline de Finalisation de Notebook

Bienvenue dans ce notebook qui met en œuvre un **pipeline générique** utilisant plusieurs agents IA spécialisés pour **finaliser automatiquement** un notebook cible :

- **CoderAgent** : se charge d’analyser et de **modifier** les cellules de code ou de texte.
- **ReviewerAgent** : **exécute** le notebook, détecte d’éventuels problèmes et formule des demandes de correction.
- **AdminAgent** : valide ou non la version finale du notebook.

### Objectif général

Nous souhaitons démontrer la capacité de ces agents à collaborer pour :
1. Analyser et modifier un notebook existant (ou fraîchement généré).
2. Exécuter et vérifier le bon fonctionnement du code.
3. Approuver ou rejeter certaines modifications, jusqu’à obtention d’un notebook conforme et **validé**.

---

Dans les prochaines sections, nous verrons comment configurer et lancer cette orchestration, jusqu’à l’approbation finale du notebook par l’AdminAgent.


## 1. Installation des dépendances

Dans cette cellule, nous installons les bibliothèques suivantes :

- **papermill** : exécuter des notebooks avec des paramètres.
- **nbformat** : lire/écrire les notebooks Jupyter.
- **ipywidgets** : créer des widgets interactifs (ici, pour l’upload).
- **semantic-kernel** : orchestrer les agents conversationnels.

> **Note :** si l'installation se fait dans un environnement actif, il peut être nécessaire de **redémarrer le kernel** après coup pour que tout soit pris en compte.


In [78]:
# Cellule Code : Installation des packages requis
%pip install papermill nbformat ipywidgets semantic-kernel --quiet


Note: you may need to restart the kernel to use updated packages.


## 2. Import des bibliothèques et configuration

Dans cette cellule, nous importons les modules Python requis tout au long de ce pipeline :

- **os, json, hashlib** : gestion de fichiers, de JSON et hachage de contenus.
- **logging** : mécanisme de journalisation (logs) personnalisables.
- **nbformat** : pour manipuler le contenu d’un notebook Jupyter.
- **papermill** : pour exécuter un notebook en Python.
- **datetime, random** : utilitaires date/heure et aléatoire.

Pour la partie IA :
- **semantic_kernel** (et sous-modules) : création et orchestration d’agents conversationnels.

Nous préparons aussi un **système de logging coloré** pour rendre la sortie plus lisible.

In [79]:
# Cellule Code : Imports et configuration du logger

import os
import json
import hashlib
import logging
import nbformat
import papermill as pm
import random
from datetime import datetime

# Imports Semantic Kernel
from semantic_kernel import Kernel
from semantic_kernel.agents import ChatCompletionAgent, AgentGroupChat
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion
from semantic_kernel.contents import ChatMessageContent, AuthorRole
from semantic_kernel.functions import kernel_function
from semantic_kernel.agents.strategies.selection.selection_strategy import SelectionStrategy
from semantic_kernel.agents.strategies.termination.termination_strategy import TerminationStrategy
from semantic_kernel.contents.function_call_content import FunctionCallContent
from semantic_kernel.contents.function_result_content import FunctionResultContent
from semantic_kernel.connectors.ai.function_choice_behavior import FunctionChoiceBehavior
from semantic_kernel.functions.kernel_arguments import KernelArguments

# Configuration du logger avec couleur
class ColorFormatter(logging.Formatter):
    colors = {
        'DEBUG': '\033[94m',
        'INFO': '\033[92m',
        'WARNING': '\033[93m',
        'ERROR': '\033[91m',
        'CRITICAL': '\033[91m\033[1m'
    }
    reset = '\033[0m'

    def format(self, record):
        msg = super().format(record)
        return f"{self.colors.get(record.levelname, '')}{msg}{self.reset}"

logger = logging.getLogger("Orchestration")
logger.setLevel(logging.DEBUG)

if not logger.handlers:
    ch = logging.StreamHandler()
    ch.setLevel(logging.DEBUG)
    formatter = ColorFormatter(
        "%(asctime)s [%(levelname)s] %(name)s - %(message)s",
        datefmt="%H:%M:%S"
    )
    ch.setFormatter(formatter)
    logger.addHandler(ch)

logger.info("Configuration initiale terminée.")


22:14:58 [INFO] Orchestration - Configuration initiale terminée.


## 3. Classe NotebookState : gestion d’état du notebook

La classe `NotebookState` encapsule la **manipulation** du notebook visé :

- **Chargement / Sauvegarde** du fichier `.ipynb`.
- **Exécution** via Papermill pour tester l’intégrité et détecter d’éventuelles erreurs.
- **Gestion d’étapes** (states) : `specified`, `implemented`, `tested`, `validated`.
- **Mises à jour ciblées** : méthode `update_cell` pour remplacer le contenu source d’une cellule.

Cette abstraction nous permet de faire évoluer le notebook au fil des interventions des différents agents (Coder, Reviewer, Admin).


In [80]:
# Cellule Code : Classe NotebookState

class NotebookState:
    """
    Gère le statut et le contenu d'un notebook.
    Les 4 états sont : 'specified', 'implemented', 'tested', 'validated'.
    """

    def __init__(self, notebook_path: str):
        self.notebook_path = notebook_path
        self._cached_notebook = None
        self._status = "specified"
        self._load_notebook()

    def _load_notebook(self):
        """Charge le notebook depuis le chemin spécifié."""
        if not os.path.exists(self.notebook_path):
            raise FileNotFoundError(f"Notebook introuvable: {self.notebook_path}")

        with open(self.notebook_path, "r", encoding="utf-8") as f:
            self._cached_notebook = nbformat.read(f, as_version=4)

        logger.debug(
            f"[NotebookState] Notebook '{self.notebook_path}' chargé, "
            f"nb de cellules={len(self._cached_notebook.cells)}."
        )

    def get_notebook_json(self) -> str:
        """Retourne le contenu actuel du notebook en mémoire au format JSON."""
        return json.dumps(self._cached_notebook, indent=2, ensure_ascii=False)

    def log_notebook_state(self, max_length: int = 10000):
        """Affiche une portion tronquée du contenu JSON du notebook."""
        notebook_json = self.get_notebook_json()
        snippet = notebook_json[:max_length]
        if len(notebook_json) > max_length:
            snippet += ">>> TRUNCATED <<<"
        logger.debug(f"[NotebookState] Current notebook state (truncated):\n{snippet}")

    def get_status(self) -> str:
        """Retourne l'état courant du notebook."""
        return self._status

    def set_status(self, new_status: str):
        """Met à jour l'état du notebook et le consigne dans les logs."""
        logger.info(f"[NotebookState] Passage de l'état {self._status} → {new_status}")
        self._status = new_status

    def save_notebook(self, path: str = ""):
        """Enregistre le notebook sur disque (par défaut, même chemin)."""
        if not path:
            path = self.notebook_path
        try:
            with open(path, "w", encoding="utf-8") as f:
                nbformat.write(self._cached_notebook, f)
            logger.info(f"[NotebookState] Notebook sauvegardé sous {path}")
        except Exception as e:
            logger.error(f"[save_notebook] Erreur de sauvegarde: {e}")
            raise

    def execute_notebook(self) -> bool:
        """
        Exécute le notebook via Papermill. Retourne True si l'exécution
        n'a pas levé d'exception, False sinon.
        """
        logger.info(f"[NotebookState] Exécution Papermill sur {self.notebook_path}.")
        # 1. Sauvegarde avant exécution
        self.save_notebook()

        success = True
        try:
            pm.execute_notebook(
                input_path=self.notebook_path,
                output_path=self.notebook_path,
                kernel_name="python3",
                progress_bar=False,
                log_output=True
            )
        except Exception as e:
            # Papermill lève une exception si une cellule a planté.
            logger.error(f"[execute_notebook] Erreur lors de l'exécution: {e}")
            success = False
        finally:
            # 3. Recharger la version avec outputs
            self._load_notebook()
            # 4. Resauvegarder pour avoir un .ipynb incluant toutes les sorties
            self.save_notebook()
            logger.info("[NotebookState] Notebook mis à jour après exécution (avec sorties).")

        return success


    def update_cell(self, cell_index: int, new_source: str):
        """Modifie le contenu source d’une cellule donnée, puis sauvegarde."""
        old_src = self._cached_notebook.cells[cell_index].source
        self._cached_notebook.cells[cell_index].source = new_source
        logger.info(f"[NotebookState] Mise à jour de la cellule {cell_index}")
        logger.debug(f"[NotebookState] Ancien contenu:\n{old_src}")
        logger.debug(f"[NotebookState] Nouveau contenu:\n{new_source}")
        self.save_notebook()

    def find_cell_indices_by_content(self, content_pattern: str):
        """Retourne la liste d'indices de cellules dont la source contient `content_pattern`."""
        indices = []
        for i, c in enumerate(self._cached_notebook.cells):
            if content_pattern in c.source:
                indices.append(i)
        return indices

    def is_approved(self) -> bool:
        """Retourne True si l'état du notebook est 'validated'."""
        return self._status == "validated"


## 4. (Optionnel) Téléverser votre propre notebook

Si vous souhaitez **remplacer** le template par un notebook personnel, utilisez le widget ci-dessous pour uploader un fichier `.ipynb`, puis exécutez la cellule suivante. Le code détectera automatiquement si vous avez uploadé un fichier, sinon il utilisera le template par défaut.


In [81]:
# Cellule Code : Widget pour uploader un notebook (optionnel)
from IPython.display import display
import ipywidgets as widgets

uploader = widgets.FileUpload(
    accept='.ipynb',
    multiple=False,
    description='Choisir notebook',
    disabled=False
)
display(uploader)

FileUpload(value=(), accept='.ipynb', description='Choisir notebook')

## 5. Création (ou chargement) du Notebook cible + Test de validité

Cette cellule va :
1. Copier un *template* interne **ou** récupérer le fichier .ipynb que vous avez éventuellement téléversé.
2. Injecter une **description de tâche** (sélectionnée aléatoirement parmi quelques exemples).
3. Exécuter automatiquement le notebook nouvellement créé via Papermill pour détecter d’éventuelles erreurs.
4. Vérifier l’intégrité et confirmer que le notebook est exécutable.

Cela nous permet de tester et valider la phase initiale de création et d’exécution du notebook.


In [82]:
# Cellule Code : Initialisation du notebook ciblé et injection de la tâche
import shutil

POSSIBLE_TASKS = [
    "Créer un notebook Python permettant de requêter DBpedia (SPARQL) et d’afficher un graphique final avec rdflib ou SPARQLWrapper et plotly. Prendre le temps de valider une requêtes dont la représentation graphique est pertinente.",
    "Créer un notebook Python qui charge le dataset Titanic depuis une URL et effectue une analyse basique. Le dataset est accessible à l'url https://raw.githubusercontent.com/datasciencedojo/datasets/refs/heads/master/titanic.csv",
    "Construire un notebook scikit-learn sur le dataset IRIS et réaliser un court modèle de classification. Le dataset est accessible à l'url https://gist.githubusercontent.com/curran/a08a1080b88344b0c8a7/raw/0e7a9b0a5d22642a06d3d5b9bcbad9890c8ee534/iris.csv"
]

def apply_task_description(notebook_state: NotebookState, task_description: str) -> bool:
    """Remplace le placeholder {{TASK_DESCRIPTION}} dans une cellule Markdown du notebook."""
    for idx, cell in enumerate(notebook_state._cached_notebook.cells):
        if cell.cell_type == "markdown" and "## 0. Objectif du Notebook" in cell.source:
            new_source = cell.source.replace(
                "{{TASK_DESCRIPTION}}",
                f"<!-- TASK-ID: {hash(task_description)} -->\n{task_description}"
            )
            notebook_state.update_cell(idx, new_source)
            return True
    return False

SOURCE_NOTEBOOK = "Notebook-Template.ipynb"
DEST_NOTEBOOK = "Notebook-Generated.ipynb"

# 1) Soit on utilise le fichier uploadé, soit on copie un template.
if uploader.value:
    fichier = uploader.value[0]
    DEST_NOTEBOOK = fichier.name
    with open(DEST_NOTEBOOK, 'wb') as f:
        f.write(fichier.content)
    logger.info(f"Notebook uploadé : {DEST_NOTEBOOK}")
else:
    shutil.copy2(SOURCE_NOTEBOOK, DEST_NOTEBOOK)
    logger.info(f"Création depuis le template : {DEST_NOTEBOOK}")

# 2) On instancie NotebookState sur ce fichier.
notebook_state = NotebookState(DEST_NOTEBOOK)

# 3) On injecte une description de tâche aléatoire.
chosen_task = random.choice(POSSIBLE_TASKS)
changed = apply_task_description(notebook_state, chosen_task)

if changed:
    logger.info("Placeholder {{TASK_DESCRIPTION}} remplacé avec succès.")
    logger.info("Exécution du notebook en place pour valider...")
    notebook_state.execute_notebook()  
    logger.info("Notebook ré-exécuté après injection de la tâche.")
else:
    logger.warning("Aucun placeholder détecté : vérifiez que la cellule Markdown contient {{TASK_DESCRIPTION}}.")

22:14:58 [INFO] Orchestration - Création depuis le template : Notebook-Generated.ipynb


22:14:58 [DEBUG] Orchestration - [NotebookState] Notebook 'Notebook-Generated.ipynb' chargé, nb de cellules=13.
22:14:58 [INFO] Orchestration - [NotebookState] Mise à jour de la cellule 1
22:14:58 [DEBUG] Orchestration - [NotebookState] Ancien contenu:
## 0. Objectif du Notebook

Dans cette section, nous décrivons l'objectif du notebook, sa fonction, les outils à utiliser et les buts à atteindre.

### Tâche originale

Voilà la tâche telle qu'elle a été initialement formulée :

{{TASK_DESCRIPTION}}


### Interprétation et sous-objectifs

Décrire ici l'interprétation de la tâche et les étapes prévues pour la réaliser.
22:14:58 [DEBUG] Orchestration - [NotebookState] Nouveau contenu:
## 0. Objectif du Notebook

Dans cette section, nous décrivons l'objectif du notebook, sa fonction, les outils à utiliser et les buts à atteindre.

### Tâche originale

Voilà la tâche telle qu'elle a été initialement formulée :

<!-- TASK-ID: 34607106656801323 -->
Créer un notebook Python permettant de requêt

## 6. Architecture de plugins : extension du NotebookState

Pour interagir avec le notebook, on définit plusieurs plugins :

- **BaseNotebookPlugin** : expose le contenu du notebook via `get_notebook_content()`.
- **CoderNotebookPlugin** : modifie le notebook (via `update_cell_by_content`) et signale la fin de l'implémentation.
- **ReviewerNotebookPlugin** : exécute le notebook, approuve ou redemande des modifications.
- **AdminNotebookPlugin** : donne la validation finale (`approve_notebook`).

Ces plugins sont **exposés** sous forme de fonctions (via `@kernel_function`) que les agents IA pourront appeler.

In [83]:
# Cellule Code : Plugins
class BaseNotebookPlugin:
    """Classe de base pour manipuler NotebookState."""
    def __init__(self, state: NotebookState):
        self.state = state

    @kernel_function(
        name="get_notebook_content",
        description="Renvoie le notebook complet au format JSON (révision courante)."
    )
    def get_notebook_content(self) -> str:
        logger.info("[BaseNotebookPlugin] get_notebook_content()")
        return self.state.get_notebook_json()

class CoderNotebookPlugin(BaseNotebookPlugin):
    @kernel_function(
        name="update_cell_by_content",
        description="Modifie la première cellule contenant 'content_pattern' (si unique)."
    )
    def update_cell_by_content(self, content_pattern: str, new_source: str) -> str:
        logger.info("[CoderNotebookPlugin] update_cell_by_content()")
        status = self.state.get_status()
        if status not in ["specified", "implemented"]:
            return f"Erreur: état actuel = '{status}' => plus de modification autorisée."

        indices = self.state.find_cell_indices_by_content(content_pattern)
        if len(indices) == 0:
            return f"Aucune cellule ne contient le motif '{content_pattern}'."
        if len(indices) > 1:
            return f"Plusieurs cellules contiennent '{content_pattern}', soyez plus précis."

        # Mise à jour
        self.state.update_cell(indices[0], new_source)
        return f"Cellule contenant '{content_pattern}' mise à jour avec succès."

    @kernel_function(
        name="finish_implementation",
        description="Marque le notebook comme 'implemented' une fois le code prêt."
    )
    def finish_implementation(self) -> str:
        logger.info("[CoderNotebookPlugin] finish_implementation()")
        status = self.state.get_status()
        if status == "specified":
            self.state.set_status("implemented")
            return "Le notebook passe en état 'implemented'."
        elif status == "implemented":
            return "Le notebook est déjà en état 'implemented'."
        else:
            return f"Impossible de passer en 'implemented' depuis l'état '{status}'."

class ReviewerNotebookPlugin(BaseNotebookPlugin):
    @kernel_function(
        name="validate_notebook",
        description="Exécute le notebook. Si approve=True => 'tested', sinon retour 'specified'."
    )
    def validate_notebook(self, approve: bool = True) -> str:
        logger.info(f"[ReviewerNotebookPlugin] validate_notebook(approve={approve})")
        status = self.state.get_status()
        if status != "implemented":
            return f"Le reviewer ne peut pas valider, état actuel = '{status}'."

        # On exécute le notebook ET on récupère le résultat
        success = self.state.execute_notebook()

        if not success:
            # Si Papermill a levé une exception, on force le retour en 'specified'
            self.state.set_status("specified")
            return (
                "Erreur détectée dans le notebook (voir les logs de Papermill). "
                "Retour à l'état 'specified' pour corrections."
            )

        # S'il n'y a pas eu d'erreur et que le reviewer veut approuver
        if approve:
            self.state.set_status("tested")
            return "Le reviewer approuve => passage à l'état 'tested'."
        else:
            self.state.set_status("specified")
            return "Le reviewer refuse => retour à l'état 'specified' pour corrections."


class AdminNotebookPlugin(BaseNotebookPlugin):
    @kernel_function(
        name="approve_notebook",
        description="Si admin_ok=True => 'validated', sinon retour 'specified'."
    )
    def approve_notebook(self, admin_ok: bool = True) -> str:
        logger.info(f"[AdminNotebookPlugin] approve_notebook(admin_ok={admin_ok})")
        status = self.state.get_status()
        if status != "tested":
            return f"Impossible d'approuver, état = '{status}' (attendu 'tested')."

        if admin_ok:
            self.state.set_status("validated")
            return "Notebook validé => état 'validated'."
        else:
            self.state.set_status("specified")
            return "Admin refuse => retour en 'specified'."

## 7. Stratégies d’orchestration

Nous définissons :

1. **ApprovedBasedTerminationStrategy** :
   - Observe l’état du notebook.
   - S’arrête si le notebook est *validé* ou si le nombre maximal d’itérations est dépassé.

2. **NotebookAwareSelectionStrategy** :
   - Décide quel agent (Coder, Reviewer, Admin) doit s’exprimer en fonction de l’état courant (`specified`, `implemented`, `tested`).
   - Arrête le flux si le notebook est déjà `validated`.

Ces stratégies permettent de **définir la logique** du tour-par-tour dans notre conversation multi-agents.

In [84]:
# Cellule Code : Définition des stratégies
from pydantic import PrivateAttr

class ApprovedBasedTerminationStrategy(TerminationStrategy):
    _state: NotebookState = PrivateAttr()
    _max_steps: int = PrivateAttr(default=10)

    def __init__(self, state: NotebookState, max_steps: int = 10):
        super().__init__()
        self._state = state
        self._max_steps = max_steps
        self._current_step = 0

    async def should_agent_terminate(self, agent, history):
        self._current_step += 1
        is_approved = self._state.is_approved()
        logger.debug(
            f"[TerminationStrategy] Step={self._current_step}/{self._max_steps}, "
            f"IsApproved={is_approved}"
        )
        if is_approved:
            logger.info("[TerminationStrategy] Notebook approuvé => arrêt.")
            return True
        if self._current_step >= self._max_steps:
            logger.warning(f"[TerminationStrategy] max_steps={self._max_steps} atteint => arrêt.")
            return True
        return False

class NotebookAwareSelectionStrategy(SelectionStrategy):
    def __init__(self, state: NotebookState):
        super().__init__()
        self._state = state
        self._index = -1

    def reset(self) -> None:
        self._index = -1

    async def select_agent(self, agents, history):
        current_status = self._state.get_status()
        logger.debug(f"[SelectionStrategy] nb_agents={len(agents)}, statut={current_status}")

        if current_status == "validated":
            logger.info("[SelectionStrategy] Notebook déjà validé => fin.")
            return None

        coder = next((a for a in agents if a.name == "CoderAgent"), None)
        reviewer = next((a for a in agents if a.name == "ReviewerAgent"), None)
        admin = next((a for a in agents if a.name == "AdminAgent"), None)

        # Choix de l'agent en fonction de l'état
        if current_status == "specified" and coder:
            toReturn = coder
        elif current_status == "implemented" and reviewer:
            toReturn = reviewer
        elif current_status == "tested" and admin:
            toReturn = admin
        else:
            logger.warning(f"[SelectionStrategy] Pas d'agent approprié pour l'état='{current_status}' => stop.")
            toReturn = None

        if toReturn:
            logger.info(f"Prochain agent sélectionné : {toReturn.name}")
        return toReturn

## 8. Création des 3 agents (Coder, Reviewer, Admin)

Nous instancions :

1. **CoderAgent** :
   - Peut appeler `update_cell_by_content` pour modifier des cellules.
   - Peut consulter le notebook complet via `get_notebook_content`.
   - Appelle `finish_implementation` lorsqu’il estime avoir fini.

2. **ReviewerAgent** :
   - Peut relire le notebook et l’exécuter via `validate_notebook(approve=...)`.
   - Si des erreurs sont détectées, il demande des corrections en passant `approve=False`.

3. **AdminAgent** :
   - Donne la validation finale avec `approve_notebook(admin_ok=...)`.
   - Peut renvoyer le notebook en phase de spécification si des points sont insatisfaisants.


In [85]:
# Cellule Code : Création des 3 agents et configuration

def create_kernel_for_agent(agent_id: str, plugin_instance) -> Kernel:
    """
    Instancie un Kernel SemanticKernel avec le plugin donné.
    """
    k = Kernel()
    k.add_service(OpenAIChatCompletion(service_id="default"))
    k.add_plugin(plugin_instance, plugin_name=f"{agent_id}_plugin")
    return k

# Plugins
coder_plugin = CoderNotebookPlugin(notebook_state)
reviewer_plugin = ReviewerNotebookPlugin(notebook_state)
admin_plugin = AdminNotebookPlugin(notebook_state)

# Kernels dédiés
coder_kernel = create_kernel_for_agent("coder_kernel", coder_plugin)
reviewer_kernel = create_kernel_for_agent("reviewer_kernel", reviewer_plugin)
admin_kernel = create_kernel_for_agent("admin_kernel", admin_plugin)

# Configuration auto-invocation des fonctions
coder_settings = coder_kernel.get_prompt_execution_settings_from_service_id("default")
coder_settings.function_choice_behavior = FunctionChoiceBehavior.Auto()

reviewer_settings = reviewer_kernel.get_prompt_execution_settings_from_service_id("default")
reviewer_settings.function_choice_behavior = FunctionChoiceBehavior.Auto()

admin_settings = admin_kernel.get_prompt_execution_settings_from_service_id("default")
admin_settings.function_choice_behavior = FunctionChoiceBehavior.Auto()

# Définition des 3 agents conversationnels
coder_agent = ChatCompletionAgent(
    service_id="coder_kernel",
    kernel=coder_kernel,
    name="CoderAgent",
    instructions=(
        "Vous êtes le **Coder**. Votre rôle :\n"
        "1) Obtenir le JSON du notebook (avec les éventuelles sorties) via get_notebook_content().\n"
        "2) Analyser le code et repérer ce qui doit être modifié ou complété.\n"
        "3) Mettre à jour le code si nécessaire via update_cell_by_content(motif, nouveau_code).\n"
        "4) Conserver les marqueurs importants dans les cellules si mentionnés.\n"
        "5) Quand vous estimez que l’implémentation est terminée, appelez finish_implementation().\n"
        "6) En cas de retours du Reviewer ou de l'Admin, réitérez : relisez à nouveau le notebook "
        "   (via get_notebook_content()) et effectuez d'autres modifications si besoin.\n"
    ),
    arguments=KernelArguments(settings=coder_settings)
)

reviewer_agent = ChatCompletionAgent(
    service_id="reviewer_kernel",
    kernel=reviewer_kernel,
    name="ReviewerAgent",
    instructions=(
        "Vous êtes le **Reviewer**.\n"
        "1) Vous devez toujours consulter le JSON complet du notebook (y compris les sorties) "
        "   via get_notebook_content() pour vérifier si des erreurs sont survenues.\n"
        "2) Uniquement après avoir consulté le notebook, appelez validate_notebook(approve=True/False) selon la qualité.\n"
        "   - Si tout est correct et que le code tourne sans erreur, passez approve=True => état='tested'.\n"
        "   - Sinon, passez approve=False => le notebook repasse en 'specified' pour corrections.\n"
        "3) Avant de prendre une décision, veillez à relire les éventuelles traces d'erreur dans les sorties.\n"
    ),
    arguments=KernelArguments(settings=reviewer_settings)
)

admin_agent = ChatCompletionAgent(
    service_id="admin_kernel",
    kernel=admin_kernel,
    name="AdminAgent",
    instructions=(
        "Vous êtes l'**Admin**.\n"
        "1) Commencez toujours par consulter le notebook et ses sorties, leur conformité et leur pertinence, via la fonction get_notebook_content().\n"
        "2) Uniquement après avoir analysé toutes les cellules, appelez approve_notebook(admin_ok=True/False).\n"
        "   - Si le résultat est complètement satisfaisant (pas d'erreurs, notebook correct, concis, éloquant), admin_ok=True\n"
        "   - Sinon admin_ok=False avec de nouvelles instructions\n"
        "3) Votre rôle est la validation finale. Vérifiez la présence éventuelle d'erreurs dans les sorties et soyez exigeants avec votre équipe d'agents. N'hésitez pas à réclamer des corrections à l'occasion de quelques aller-retours de finalisation.\n"
    ),
    arguments=KernelArguments(settings=admin_settings)
)

# GroupChat : on associe les stratégies
termination_strategy = ApprovedBasedTerminationStrategy(notebook_state)
selection_strategy = NotebookAwareSelectionStrategy(notebook_state)

group_chat = AgentGroupChat(
    agents=[coder_agent, reviewer_agent, admin_agent],
    selection_strategy=selection_strategy,
    termination_strategy=termination_strategy
)

logger.info("Agents créés et group_chat initialisé avec instructions mises à jour.")


22:15:01 [INFO] Orchestration - Agents créés et group_chat initialisé avec instructions mises à jour.


## 9. Boucle de conversation

Enfin, nous lançons la **conversation multi-agents**. Les points clés :

- Chaque itération, on laisse un agent parler (sélectionné par `NotebookAwareSelectionStrategy`).
- Les agents peuvent **s’auto-invoquer** des fonctions plugin (e.g. `update_cell_by_content`) selon leur rôle.
- La boucle se termine si :
  1. Le notebook est validé (`is_approved() == True`).
  2. Le nombre de tours dépasse le maximum défini.

Vous verrez dans la sortie (logs) une trace de :
1. **Quel agent** s’exprime.
2. **Les appels de fonctions** internes.
3. **Les résultats** de ces appels.
4. Les logs de papermill si des erreurs sont détectées.


In [86]:
# Cellule Code : Lancement de la conversation async

import asyncio

async def run_conversation():
    """Lance la conversation multi-agents et affiche les étapes."""
    try:
        logger.info("Version initiale du notebook :")
        notebook_state.log_notebook_state()

        # Ajout d'un message initial dans l'historique pour cadrer la conversation
        initial_content = notebook_state.get_notebook_json()
        group_chat.history.add_system_message(f"NOTEBOOK CONTENT:\n{initial_content}")
        group_chat.history.add_user_message(
            "Bonjour, j'aimerais finaliser ce notebook et corriger ses cellules si besoin."
        )

        iteration = 0
        async for message in group_chat.invoke():
            iteration += 1
            role = message.role.name
            content = message.content
            logger.info(f"{role.upper()} >>> {content}")

            # Arrêt si le notebook est validé
            if notebook_state.is_approved():
                logger.info("Notebook approuvé => fin de la conversation.")
                break

        logger.info("Version finale du notebook, après la conversation :")
        notebook_state.log_notebook_state(max_length=20000)

    except Exception as e:
        logger.error(f"Erreur inattendue: {str(e)}")
    finally:
        logger.info(f"Statut final - Approuvé: {notebook_state.is_approved()}")
        logger.info("Conversation terminée.")

await run_conversation()


22:15:01 [INFO] Orchestration - Version initiale du notebook :
22:15:01 [DEBUG] Orchestration - [NotebookState] Current notebook state (truncated):
{
  "cells": [
    {
      "cell_type": "markdown",
      "id": "516d2854",
      "metadata": {
        "papermill": {
          "duration": 0.0032,
          "end_time": "2025-02-14T21:15:00.313211",
          "exception": false,
          "start_time": "2025-02-14T21:15:00.310011",
          "status": "completed"
        },
        "tags": []
      },
      "source": "# Notebook de travail\n\nCe notebook est généré de façon incrémentale pour accomplir la tâche décrite ci-dessous."
    },
    {
      "cell_type": "markdown",
      "id": "2a226db6",
      "metadata": {
        "papermill": {
          "duration": 0.001881,
          "end_time": "2025-02-14T21:15:00.318198",
          "exception": false,
          "start_time": "2025-02-14T21:15:00.316317",
          "status": "completed"
        },
        "tags": []
      },
      "source"